<a href="https://colab.research.google.com/github/EmperoR1127/CSI5155_project/blob/master/csi5155_assignment_2_part_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
from scipy.io import arff
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "/content/drive/My Drive/Images/"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [0]:
#load four dataset
seismic_bumps_data = pd.DataFrame(arff.loadarff('/content/drive/My Drive/Data/seismic-bumps.arff')[0])
labor_neg_data = pd.read_csv("/content/drive/My Drive/Data/labor-neg.csv")
iris_data = pd.read_csv("/content/drive/My Drive/Data/iris.csv")
voting_data = pd.read_csv("/content/drive/My Drive/Data/voting-records.csv")

In [55]:
#process the seismic_bumps_data
#pre-process the train_set
sb_train_labels = seismic_bumps_data[["class"]].copy()
sb_train_set = seismic_bumps_data.drop(["class"], axis=1)
sb_train_labels["class"] = sb_train_labels["class"].map(lambda x: str(x)[2])
sb_train_set_num = seismic_bumps_data.drop(["seismic","seismoacoustic","shift", "ghazard", "class"], axis=1)
sb_train_set_cat = seismic_bumps_data.drop(["genergy","gpuls","gdenergy", "gdpuls", "nbumps", "nbumps2", "nbumps3", "nbumps4", "nbumps5", "nbumps6", "nbumps7", "nbumps89", "energy", "maxenergy", "class"], axis=1)
#build the pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
sb_num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),('std_scaler', StandardScaler()),])
sb_full_pipeline = ColumnTransformer([("num", sb_num_pipeline, list(sb_train_set_num)),("cat", OneHotEncoder(), list(sb_train_set_cat)),])
#prepare the data
sb_train_set_prepared = sb_full_pipeline.fit_transform(sb_train_set)
#prepare the target
sb_encoder = LabelEncoder()
sb_train_labels_prepared = sb_encoder.fit_transform(sb_train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
#process the labor_neg_data
ln_train_labels = labor_neg_data['class'].copy()
ln_train_set = labor_neg_data.drop(["class"], axis = 1)
ln_train_set_num = ln_train_set.drop(["cola","pension","educ_allw", "vacation", "lngtrm_disabil", "dntl_ins", "bereavement", "empl_hplan"], axis=1)
ln_train_set_cat = ln_train_set.drop(["dur","wage1","wage2", "wage3", "hours", "stby_pay", "shift_diff", "holidays"], axis=1)
#build the pipeline
ln_num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),('std_scaler', StandardScaler()),])
ln_full_pipeline = ColumnTransformer([("num", ln_num_pipeline, list(ln_train_set_num)),("cat", OneHotEncoder(), list(ln_train_set_cat)),])
#prepare the data
ln_train_set_prepared = ln_full_pipeline.fit_transform(ln_train_set)
#prepare the target
ln_encoder = LabelEncoder()
ln_train_labels_prepared = ln_encoder.fit_transform(ln_train_labels)

In [0]:
#process the iris_data
ir_train_labels = iris_data['class'].copy()
ir_train_set = iris_data.drop(["class"], axis = 1)
#build the pipeline
ir_num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),('std_scaler', StandardScaler()),])
ir_full_pipeline = ColumnTransformer([("num", ir_num_pipeline, list(ir_train_set)),])
#prepare the data
ir_train_set_prepared = ir_full_pipeline.fit_transform(ir_train_set)
#prepare the target
ir_encoder = LabelEncoder()
ir_train_labels_prepared = ir_encoder.fit_transform(ir_train_labels)

In [0]:
#process the voting_data
vd_train_labels = voting_data['Class'].copy()
vd_train_set = voting_data.drop(["Class"], axis = 1)
#build the pipeline
vd_full_pipeline = ColumnTransformer([("cat", OneHotEncoder(), list(vd_train_set)),])
#prepare the data
vd_train_set_prepared = vd_full_pipeline.fit_transform(vd_train_set)
#prepare the target
vd_encoder = LabelEncoder()
vd_train_labels_prepared = vd_encoder.fit_transform(vd_train_labels)

In [63]:
#train the model using DECISION TREE algorithm with 10 fold cross validation
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
score_method = make_scorer(accuracy_score)
dt_clf = DecisionTreeClassifier()
print("The accuracies of the DECISION TREE algorithm trained against seismic_bumps_data are: ")
print(cross_val_score(dt_clf, sb_train_set_prepared, sb_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the DECISION TREE algorithm trained against labor_neg_data are: ")
print(cross_val_score(dt_clf, ln_train_set_prepared, ln_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the DECISION TREE algorithm trained against iris_data are: ")
print(cross_val_score(dt_clf, ir_train_set_prepared, ir_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the DECISION TREE algorithm trained against voting_data are: ")
print(cross_val_score(dt_clf, vd_train_set_prepared, vd_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the DECISION TREE algorithm trained against seismic_bumps_data are: 
[0.86100386 0.63706564 0.75289575 0.86100386 0.87596899 0.89147287
 0.86434109 0.91472868 0.89147287 0.91860465]
The accuracies of the DECISION TREE algorithm trained against labor_neg_data are: 
[1.         0.83333333 1.         0.66666667 1.         0.5
 0.83333333 0.8        0.8        1.        ]
The accuracies of the DECISION TREE algorithm trained against iris_data are: 
[1.         0.93333333 1.         0.93333333 0.93333333 0.86666667
 0.93333333 0.93333333 1.         1.        ]
The accuracies of the DECISION TREE algorithm trained against voting_data are: 
[0.95454545 0.97727273 0.97727273 1.         0.97727273 0.95454545
 0.95454545 0.90697674 0.88095238 0.95238095]


In [64]:
#train the model using K NEAREST NEIGHBOURS algorithm with 10 fold cross validation
from sklearn import neighbors
n_neighbors = 5
knn_clf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
print("The accuracies of the K NEAREST NEIGHBOURS algorithm trained against seismic_bumps_data are: ")
print(cross_val_score(knn_clf, sb_train_set_prepared, sb_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the K NEAREST NEIGHBOURS algorithm trained against labor_neg_data are: ")
print(cross_val_score(knn_clf, ln_train_set_prepared, ln_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the K NEAREST NEIGHBOURS algorithm trained against iris_data are: ")
print(cross_val_score(knn_clf, ir_train_set_prepared, ir_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the K NEAREST NEIGHBOURS algorithm trained against voting_data are: ")
print(cross_val_score(knn_clf, vd_train_set_prepared, vd_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the K NEAREST NEIGHBOURS algorithm trained against seismic_bumps_data are: 
[0.89189189 0.8030888  0.87644788 0.93436293 0.93023256 0.91472868
 0.93023256 0.93023256 0.93410853 0.9379845 ]
The accuracies of the K NEAREST NEIGHBOURS algorithm trained against labor_neg_data are: 
[1.         0.83333333 1.         1.         1.         1.
 1.         0.8        0.8        1.        ]
The accuracies of the K NEAREST NEIGHBOURS algorithm trained against iris_data are: 
[1.         0.93333333 1.         0.93333333 0.86666667 0.93333333
 0.86666667 1.         1.         1.        ]
The accuracies of the K NEAREST NEIGHBOURS algorithm trained against voting_data are: 
[0.97727273 0.90909091 0.93181818 0.86363636 0.95454545 0.90909091
 0.95454545 0.95348837 0.83333333 0.92857143]


In [65]:
#train the model using NAIVE BAYES algorithm with 10 fold cross validation
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()
print("The accuracies of NAIVE BAYES algorithm trained against seismic_bumps_data are: ")
print(cross_val_score(gnb_clf, sb_train_set_prepared, sb_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of NAIVE BAYES algorithm trained against labor_neg_data are: ")
print(cross_val_score(gnb_clf, ln_train_set_prepared, ln_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of NAIVE BAYES algorithm trained against iris_data are: ")
print(cross_val_score(gnb_clf, ir_train_set_prepared, ir_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of NAIVE BAYES algorithm trained against voting_data are: ")
print(cross_val_score(gnb_clf, vd_train_set_prepared, vd_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of NAIVE BAYES algorithm trained against seismic_bumps_data are: 
[0.48648649 0.31660232 0.31660232 0.85328185 0.39147287 0.25968992
 0.07751938 0.36046512 0.4379845  0.3255814 ]
The accuracies of NAIVE BAYES algorithm trained against labor_neg_data are: 
[1.         1.         1.         0.83333333 1.         1.
 1.         0.8        0.6        1.        ]
The accuracies of NAIVE BAYES algorithm trained against iris_data are: 
[0.93333333 0.93333333 1.         0.93333333 0.93333333 0.93333333
 0.86666667 1.         1.         1.        ]
The accuracies of NAIVE BAYES algorithm trained against voting_data are: 
[0.97727273 0.93181818 0.97727273 0.93181818 0.95454545 0.90909091
 1.         1.         0.80952381 0.95238095]


In [66]:
#train the model using RULE BASED algorithm with 10 fold cross validation
from sklearn.dummy import DummyClassifier
dc_clf = DummyClassifier(strategy='stratified')
print("The accuracies of RULE BASED algorithm trained against seismic_bumps_data are: ")
print(cross_val_score(dc_clf, sb_train_set_prepared, sb_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of RULE BASED algorithm trained against labor_neg_data are: ")
print(cross_val_score(dc_clf, ln_train_set_prepared, ln_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of RULE BASED algorithm trained against iris_data are: ")
print(cross_val_score(dc_clf, ir_train_set_prepared, ir_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of RULE BASED algorithm trained against voting_data are: ")
print(cross_val_score(dc_clf, vd_train_set_prepared, vd_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of RULE BASED algorithm trained against seismic_bumps_data are: 
[0.88416988 0.84169884 0.89189189 0.86486486 0.87596899 0.87209302
 0.87209302 0.87984496 0.89534884 0.86046512]
The accuracies of RULE BASED algorithm trained against labor_neg_data are: 
[0.33333333 0.33333333 0.33333333 0.33333333 0.5        0.66666667
 0.5        0.4        0.4        0.4       ]
The accuracies of RULE BASED algorithm trained against iris_data are: 
[0.33333333 0.33333333 0.46666667 0.33333333 0.33333333 0.33333333
 0.33333333 0.33333333 0.33333333 0.46666667]
The accuracies of RULE BASED algorithm trained against voting_data are: 
[0.52272727 0.56818182 0.56818182 0.56818182 0.43181818 0.56818182
 0.43181818 0.39534884 0.4047619  0.45238095]
